In [1]:
import joblib
import gc
import multiprocessing as mp
import numpy as np
import pandas as pd
from datetime import datetime
import time
from pandas import HDFStore
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from operator import itemgetter
import matplotlib.pyplot as plt
# from bayes_opt import BayesianOptimization

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import check_cv, train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, log_loss, auc

In [2]:
files = ['lgb_stack1', 'lgb_stack2', 'lgb_stack3']
sub_item = pd.concat([pd.read_csv("data/{}.csv".format(d)) for d in files])

In [3]:
sub_item = sub_item.groupby(['order_id','product_id']).yhat.mean().reset_index()

In [4]:
sub_item.yhat.min(), sub_item.yhat.max()

(0.0005500177181180012, 0.9518874461148816)

In [5]:
sub = sub_item.groupby('order_id').product_id.apply(list).to_frame()
sub['yhat'] = sub_item.groupby('order_id').yhat.apply(list)

In [8]:
"""
@author: Faron
"""
'''
This kernel implements the O(n²) F1-Score expectation maximization algorithm presented in
"Ye, N., Chai, K., Lee, W., and Chieu, H.  Optimizing F-measures: A Tale of Two Approaches. In ICML, 2012."
It solves argmax_(0 <= k <= n,[[None]]) E[F1(P,k,[[None]])]
with [[None]] being the indicator for predicting label "None"
given posteriors P = [p_1, p_2, ... , p_n], where p_1 > p_2 > ... > p_n
under label independence assumption by means of dynamic programming in O(n²).
'''
class F1Optimizer():
    def __init__(self):
        pass

    @staticmethod
    def get_expectations(P, pNone=None):
        expectations = []
        P = np.sort(P)[::-1]

        n = np.array(P).shape[0]
        DP_C = np.zeros((n + 2, n + 1))
        if pNone is None:
            pNone = (1.0 - P).prod()

        DP_C[0][0] = 1.0
        for j in range(1, n):
            DP_C[0][j] = (1.0 - P[j - 1]) * DP_C[0, j - 1]

        for i in range(1, n + 1):
            DP_C[i, i] = DP_C[i - 1, i - 1] * P[i - 1]
            for j in range(i + 1, n + 1):
                DP_C[i, j] = P[j - 1] * DP_C[i - 1, j - 1] + (1.0 - P[j - 1]) * DP_C[i, j - 1]

        DP_S = np.zeros((2 * n + 1,))
        DP_SNone = np.zeros((2 * n + 1,))
        for i in range(1, 2 * n + 1):
            DP_S[i] = 1. / (1. * i)
            DP_SNone[i] = 1. / (1. * i + 1)
        for k in range(n + 1)[::-1]:
            f1 = 0
            f1None = 0
            for k1 in range(n + 1):
                f1 += 2 * k1 * DP_C[k1][k] * DP_S[k + k1]
                f1None += 2 * k1 * DP_C[k1][k] * DP_SNone[k + k1]
            for i in range(1, 2 * k - 1):
                DP_S[i] = (1 - P[k - 1]) * DP_S[i] + P[k - 1] * DP_S[i + 1]
                DP_SNone[i] = (1 - P[k - 1]) * DP_SNone[i] + P[k - 1] * DP_SNone[i + 1]
            expectations.append([f1None + 2 * pNone / (2 + k), f1])

        return np.array(expectations[::-1]).T

    @staticmethod
    def maximize_expectation(P, pNone=None):
        expectations = F1Optimizer.get_expectations(P, pNone)

        ix_max = np.unravel_index(expectations.argmax(), expectations.shape)
        max_f1 = expectations[ix_max]

        predNone = True if ix_max[0] == 0 else False
        best_k = ix_max[1]

        return best_k, predNone, max_f1

    @staticmethod
    def _F1(tp, fp, fn):
        return 2 * tp / (2 * tp + fp + fn)

    @staticmethod
    def _Fbeta(tp, fp, fn, beta=1.0):
        beta_squared = beta ** 2
        return (1.0 + beta_squared) * tp / ((1.0 + beta_squared) * tp + fp + beta_squared * fn)

######################################################################

def get_best_prediction(items, preds, pNone=None):
#    print("Maximize F1-Expectation")
#    print("=" * 23)
    items_preds = sorted(list(zip(items, preds)), key=itemgetter(1), reverse=True)
    P = [p for i,p in items_preds]
    L = [i for i,p in items_preds]
    
    opt = F1Optimizer.maximize_expectation(P, pNone)
    best_prediction = ['None'] if opt[1] else []
    best_prediction += (L[:opt[0]])
#    f1_max = opt[2]
    
#    print("Prediction {} yields best E[F1] of {}\n".format(best_prediction, f1_max))
    return ' '.join(list(map(str,best_prediction)))

def multi(i):
    if i%1000==0:
        print('{} -> {:.3f} min'.format(i, (time.time()-st_time)/60))
    items = sub.loc[i,'product_id']
    preds = sub.loc[i,'yhat']
    ret = get_best_prediction(items, preds)
    return ret

In [9]:
st_time = time.time()
sub.reset_index(inplace=True)
pool = mp.Pool(70)
callback = pool.map(multi, range(sub.shape[0]))
sub['products'] = callback

0 -> 0.016 min
11000 -> 0.082 min
3000 -> 0.541 min
18000 -> 0.714 min
7000 -> 0.931 min
10000 -> 1.141 min
6000 -> 1.312 min
2000 -> 1.752 min
14000 -> 1.815 min
17000 -> 1.917 min
13000 -> 2.082 min
9000 -> 2.086 min
16000 -> 2.624 min
5000 -> 2.973 min
12000 -> 3.164 min
1000 -> 3.300 min
22000 -> 3.850 min
4000 -> 3.963 min
26000 -> 3.969 min
8000 -> 4.061 min
15000 -> 4.319 min
21000 -> 4.609 min
25000 -> 4.685 min
29000 -> 4.741 min
33000 -> 5.078 min
24000 -> 5.540 min
37000 -> 5.600 min
28000 -> 5.724 min
19000 -> 6.170 min
23000 -> 6.339 min
36000 -> 6.401 min
32000 -> 6.448 min
20000 -> 6.580 min
27000 -> 6.671 min
31000 -> 6.699 min
35000 -> 7.091 min
30000 -> 8.151 min
44000 -> 8.402 min
48000 -> 8.406 min
34000 -> 8.631 min
52000 -> 8.645 min
40000 -> 8.687 min
39000 -> 8.790 min
43000 -> 9.175 min
38000 -> 9.441 min
47000 -> 9.568 min
42000 -> 10.029 min
51000 -> 10.250 min
46000 -> 10.532 min
55000 -> 10.652 min
54000 -> 11.066 min
41000 -> 11.256 min
45000 -> 11.272 min

In [10]:
sub[['order_id', 'products']].to_csv("data/bagging_submission.csv", index=False)